In [1]:
import os
import pandas as pd
import sys
import time

from PIL import Image
from sqlalchemy import create_engine, func, text
from sqlalchemy.orm import sessionmaker

sys.path.append('../..')
sys.path.append('../../../ajna_docs/commons')


In [2]:
from ajna_commons.flask.conf import SQL_URI
from ajna_commons.utils.images import mongo_image
from virasana.db import mongodb as db


2025-09-12 12:07 ajna         INFO     Configuração de log efetuada


2025-09-12 12:07:53,710 ajna         INFO     Configuração de log efetuada


In [3]:
%load_ext autoreload 
%autoreload 2
engine = create_engine(SQL_URI)

# Recuperar informações de REDEX e OPERADORES

In [4]:
# Constantes organizando códigos
LISTA_OPERADORES = "('8931356', '8931359', '8931404', '8931318')"  # Terminais exportadores BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO
LISTA_REDEX = "('8931309', '8933204')" # Redex da Localfrio/Movecta e Redex da Santos Brasil Clia
FILTRO_LIKE_REDEX = "'89327%'"  # Abrange todos os outros Redex santistas

from datetime import datetime, timedelta

agora = datetime.now()
ha_X_dias = agora - timedelta(days=3)

# Parâmetros de data
data_final = agora.strftime('%Y-%m-%d %H:%M:%S')
data_inicial = ha_X_dias.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
# SQL para entrada_operadores (alias e)
sql_entrada_operadores = f"""
SELECT 
    id, placa, numeroConteiner, codigoRecinto, dataHoraOcorrencia, operacao, direcao, 
    cnpjTransportador, cpfMotorista, nomeMotorista, listaNfe, numeroConhecimento
FROM apirecintos_acessosveiculo
WHERE 
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'E'
    AND codigoRecinto IN {LISTA_OPERADORES}
    AND numeroConteiner !='' 
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'E' é porque queremos a Entrada no Operador Terminal Exportador (BTP, SANTOSBRASIL, DPW/EMBRAPORT e ECOPORTO)
# Este 'C' é porque é registro de acesso (entrada) e não agendamento

In [6]:
# SQL para pesagem_operador - pesagem com recintos dos operadores (terminais exportadores)
sql_pesagem_operador = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND codigoRecinto IN {LISTA_OPERADORES}
"""
# Se o caminhao com conteiner pesa 22 toneladas ou mais, então, ele está carregado (não é vazio)

In [7]:
# SQL para pesagem_redex - pesagem com recintos Redex (estufadores)
sql_pesagem_redex = f"""
SELECT 
    id, placa, placaSemirreboque, pesoBrutoBalanca, numeroConteiner, codigoRecinto, dataHoraOcorrencia
FROM apirecintos_pesagensveiculo
WHERE 
    pesoBrutoBalanca >= 22000
    AND dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
"""
# Se o caminhao com conteiner pesa 22 toneladas ou mais, então, ele está carregado (não é vazio)

In [8]:
# SQL para saida_redex (alias c)
sql_saida_redex = f"""
SELECT 
    id, placa, numeroConteiner, codigoRecinto, dataHoraOcorrencia, direcao
FROM apirecintos_acessosveiculo
WHERE
    dataHoraOcorrencia BETWEEN :data_inicial AND :data_final
    AND operacao = 'C'
    AND direcao = 'S'
    AND (codigoRecinto IN {LISTA_REDEX} OR codigoRecinto LIKE {FILTRO_LIKE_REDEX})
    AND numeroConteiner !=''
"""
# Este símbolo != '' Significa que o numero do conteiner NÃO pode ser nulo porque queremos caminão com conteiner
# Este 'S' é porque queremos a Saida do Redex estufador
# Este 'C' é porque é registro de acesso (saída) e não agendamento

In [9]:
# Execução das consultas com parâmetros
df_entrada_operadores = pd.read_sql(
    text(sql_entrada_operadores), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [10]:
df_entrada_operadores

,id,placa,numeroConteiner,codigoRecinto,dataHoraOcorrencia,operacao,direcao,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento
0,12217112,DPE7E30,EISU9443695,8931318,2025-09-09 12:14:14,C,E,12014047000193,63262126691,AMILTON GONCALVES AMANCIO,,
1,12217110,CUA0J11,MRKU7836402,8931318,2025-09-09 12:14:58,C,E,21425093000176,39485089848,STEFANO FERNANDO A GELUMBAUS MENDON,,152505269091920
2,12217115,KJQ0F31,EMCU5379168,8931318,2025-09-09 12:22:17,C,E,12014047000193,08923508810,PEDRO DE SOUZA,,152505260578540
3,12217121,EFV4C14,MRSU8695256,8931318,2025-09-09 12:23:29,C,E,12014047000193,31433236800,LEANDRO DA SILVA,,
4,12217119,OYM5143,MRSU8818330,8931318,2025-09-09 12:24:20,C,E,21425093000176,19281546892,JORGE WILLIANS AFONSO CHAGAS,,
...,...,...,...,...,...,...,...,...,...,...,...,...
11181,12296698,MJO9I50,TLLU5201283,8931404,2025-09-11 23:32:43,C,E,42548097000111,16960826845,SILVIO APARECIDO DA SILVA,,
11182,12296721,AAW2B18,ONEU9398611,8931404,2025-09-11 23:39:33,C,E,21425093000176,37339188860,RONALDO LUIZ BARROS SOUSA,,
11183,12296722,RNR3I42,GESU1213432,8931404,2025-09-11 23:39:40,C,E,31962717000148,29710532880,CRISTIANO WILLIANS VIEIRA BRAS,35250908299558000121550010000002051383403952,
11184,12296741,CUA6485,FANU1986938,8931404,2025-09-11 23:48:18,C,E,45050663000159,21569023808,MARCELO DA SILVA MOURA,,


In [11]:
df_saida_redex = pd.read_sql(
    text(sql_saida_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

In [12]:
df_saida_redex

,id,placa,numeroConteiner,codigoRecinto,dataHoraOcorrencia,direcao
0,12229906,SSR8H65,SEGU9041015,8932761,2025-09-09 12:12:13,S
1,12230808,DVS2C47,EGSU9426518,8932796,2025-09-09 12:17:43,S
2,12230575,DVS2C47,EGSU9426518,8932796,2025-09-09 12:22:38,S
3,12229910,TLC6A63,MNBU4426195,8932761,2025-09-09 12:27:20,S
4,12229909,TLM1H57,CGMU5804679,8932761,2025-09-09 12:28:00,S
...,...,...,...,...,...,...
1107,12299613,BXF4E94,MEDU5888617,8932797,2025-09-11 23:36:02,S
1108,12299612,DPC9B40,MEDU5443914,8932797,2025-09-11 23:36:39,S
1109,12297971,SWU1E80,SEGU9994172,8932761,2025-09-11 23:37:29,S
1110,12299210,MIH1F29,TCKU1854406,8932796,2025-09-11 23:42:26,S


In [13]:
df_saidas_entradas = pd.merge(df_entrada_operadores, df_saida_redex, how='inner', on=['placa', 'numeroConteiner'] , suffixes=('_E_Operador', '_S_REDEX'))
# Este 'inner' faz com que a tabela final obtida saidas_entradas só tenham dados que tem em Entrada Operador e Saída Rede
# Dados que não tenham ligação correspondente (são nulos) NÃO aparecerão
# Se você quiser que os não correspondentes também apareçam troque INNER por LEFT

In [14]:
# df_saidas_entradas['transitTime'] = (df_saidas_entradas['dataHoraOcorrencia_E_Operador'] - 
                                    # df_saidas_entradas['dataHoraOcorrencia_S_REDEX']).dt.total_seconds() / 3600


In [15]:
df_saidas_entradas

,id_E_Operador,placa,numeroConteiner,codigoRecinto_E_Operador,dataHoraOcorrencia_E_Operador,operacao,direcao_E_Operador,cnpjTransportador,cpfMotorista,nomeMotorista,listaNfe,numeroConhecimento,id_S_REDEX,codigoRecinto_S_REDEX,dataHoraOcorrencia_S_REDEX,direcao_S_REDEX
0,12220622,NCP8J86,PCIU9088970,8931356,2025-09-09 14:07:36,C,E,58130089000786,28938218805,RODRIGO SEABRA DA SILVA,21250989096457004304550010000663861746054063,,12230821,8932796,2025-09-09 13:21:03,S
1,12220671,DVS2C47,EGSU9426518,8931356,2025-09-09 14:09:23,C,E,58130089000786,31328693848,ANDERSON BUENO CAMPOS,35250847067525011142550250000365711889755499,,12230808,8932796,2025-09-09 12:17:43,S
2,12220671,DVS2C47,EGSU9426518,8931356,2025-09-09 14:09:23,C,E,58130089000786,31328693848,ANDERSON BUENO CAMPOS,35250847067525011142550250000365711889755499,,12230575,8932796,2025-09-09 12:22:38,S
3,12220735,DBC7116,PCIU9541048,8931356,2025-09-09 14:32:48,C,E,58130089000786,26696291806,CLAUDY REGIS CARVALHO REIS,21250989096457004304550010000663861746054063,,12230828,8932796,2025-09-09 13:26:32,S
4,12220735,DBC7116,PCIU9541048,8931356,2025-09-09 14:32:48,C,E,58130089000786,26696291806,CLAUDY REGIS CARVALHO REIS,21250989096457004304550010000663861746054063,,12230831,8932796,2025-09-09 13:36:55,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
782,12296433,SWO3C33,HLXU1061105,8931404,2025-09-11 22:12:22,C,E,51641405000140,28481410934,JORGE BOEIRA DOS SANTOS,35250945736992000158550000000240521547005889,,12262436,8932722,2025-09-10 22:37:54,S
783,12296598,NTO9D51,HAMU3316213,8931404,2025-09-11 22:58:02,C,E,58130089000786,39914451829,GLEIDSON JUNIOR SANTOS,,,12299824,8932798,2025-09-11 21:27:01,S
784,12296606,FQN3763,HAMU2437416,8931404,2025-09-11 22:59:08,C,E,58130089000786,37889811802,MIGUEL RUIZ DOS SANTOS,,,12299821,8932798,2025-09-11 20:51:12,S
785,12296617,ASX1H52,HAMU3055906,8931404,2025-09-11 23:02:20,C,E,58130089000786,00113214588,JORGE CASCIO SOUZA SANTOS SANT,,,12299705,8932798,2025-09-11 21:22:45,S


In [16]:
df_saidas_entradas.to_excel('saidas_entradas.xlsx')

In [17]:
df_pesagem_operador = pd.read_sql(
    text(sql_pesagem_operador), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)

df_pesagem_redex = pd.read_sql(
    text(sql_pesagem_redex), engine, params={"data_inicial": data_inicial, "data_final": data_final}, parse_dates=['dataHoraOcorrencia']
)



# Seguem os merges e filtros conforme a lógica desejada especificada anteriormente
# Por exemplo, aqui unimos pesagem_redex com saida_redex e depois com entrada_operadores conforme passo a passo
df_pesagem_saida = pd.merge(df_pesagem_redex, df_saida_redex, on=['placa', 'codigoRecinto', 'numeroConteiner'], suffixes=('_pesagem', '_saida'))

df_pesagem_saida = df_pesagem_saida[
    (df_pesagem_saida['dataHoraOcorrencia_pesagem'] <= df_pesagem_saida['dataHoraOcorrencia_saida']) &
    ((df_pesagem_saida['dataHoraOcorrencia_saida'] - df_pesagem_saida['dataHoraOcorrencia_pesagem']).dt.days <= 3) &
    (df_pesagem_saida['direcao'] == 'S')
]

df_final = pd.merge(df_pesagem_saida, df_entrada_operadores, left_on=['placa', 'numeroConteiner'], right_on=['placa', 'numeroConteiner'], suffixes=('', '_entrada'))

df_final = df_final[
    ((df_final['dataHoraOcorrencia'] - df_final['dataHoraOcorrencia_saida']).dt.days <= 3) &
    (df_final['codigoRecinto'] != df_final['codigoRecinto_entrada']) &
]

df_final = df_final.drop_duplicates()

# Seleção de colunas para output
output_cols = [
    'dataHoraOcorrencia_pesagem', 'id_pesagem', 'placa', 'placaSemirreboque', 'pesoBrutoBalanca', 'numeroConteiner', 'codigoRecinto_pesagem',
    'dataHoraOcorrencia_saida', 'placa', 'codigoRecinto_saida', 'direcao', 'numeroConteiner',
    'dataHoraOcorrencia', 'placa', 'id', 'cnpjTransportador', 'cpfMotorista', 'nomeMotorista', 'codigoRecinto', 'operacao', 'direcao', 'numeroConteiner', 'listaNfe', 'numeroConhecimento'
]

df_result = df_final[output_cols]

print(df_result.head())

SyntaxError: invalid syntax (1004912324.py, line 26)